In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import logging
import time
import random
from sqlalchemy import create_engine, Column, String, Integer, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

logging.basicConfig(level=logging.INFO)

Base = declarative_base()

class Artist(Base):
    __tablename__ = 'artists'
    id = Column(String, primary_key=True)
    name = Column(String)

class Album(Base):
    __tablename__ = 'albums'
    id = Column(String, primary_key=True)
    name = Column(String)
    release_date = Column(String)
    artist_id = Column(String, ForeignKey('artists.id'))
    artist = relationship("Artist")

class Track(Base):
    __tablename__ = 'tracks'
    id = Column(String, primary_key=True)
    name = Column(String)
    popularity = Column(Integer)
    duration_ms = Column(Integer)
    album_id = Column(String, ForeignKey('albums.id'))
    album = relationship("Album")

def robust_fetch(func):
    def wrapper(*args, **kwargs):
        attempts = 0
        while attempts < 5:
            try:
                return func(*args, **kwargs)
            except spotipy.exceptions.SpotifyException as e:
                logging.error(f"Spotify API error: {e}")
                time.sleep((2 ** attempts) + random.random()) 
                attempts += 1
            except Exception as e:
                logging.error(f"Unexpected error: {e}")
                break
        return None
    return wrapper

@robust_fetch
def fetch_artist_top_tracks(sp, artist_uri):
    results = sp.artist_top_tracks(artist_uri)
    tracks_data = []
    for track in results['tracks'][:50]:
        track_info = {
            'track_id': track['id'],
            'name': track['name'],
            'popularity': track['popularity'],
            'duration_ms': track['duration_ms'],
            'album_id': track['album']['id'],
            'album_name': track['album']['name'],
            'release_date': track['album']['release_date'],
            'artist_id': track['artists'][0]['id'],
            'artist_name': track['artists'][0]['name']
        }
        tracks_data.append(track_info)
    return tracks_data

def main():
    client_id = '5b2023b50cd44ccca291f436252f1381'
    client_secret = 'b87bc93755134e1e97bf139ca8855ca7'
    artist_uris = {
        'Taylor Swift': 'spotify:artist:06HL4z0CvFAxyc27GXpf02',
        'Bad Bunny': 'spotify:artist:4q3ewBCX7sLwd24euuV69X',
        'The Weeknd': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ',
        'Drake': 'spotify:artist:3TVXtAsR1Inumwj472S9r4',
        'Peso Pluma': 'spotify:artist:12GqGscKJx3aE4t07u7eVZ'
    }

    credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=credentials)

    engine = create_engine('sqlite:///spotify_tracks.db')
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()

    for artist_name, artist_uri in artist_uris.items():
        tracks_data = fetch_artist_top_tracks(sp, artist_uri)
        for data in tracks_data:
            artist = session.query(Artist).filter_by(id=data['artist_id']).first()
            if not artist:
                artist = Artist(id=data['artist_id'], name=data['artist_name'])
                session.add(artist)

            album = session.query(Album).filter_by(id=data['album_id']).first()
            if not album:
                album = Album(id=data['album_id'], name=data['album_name'], release_date=data['release_date'], artist=artist)
                session.add(album)

            track = session.query(Track).filter_by(id=data['track_id']).first()
            if not track:
                track = Track(id=data['track_id'], name=data['name'], popularity=data['popularity'], duration_ms=data['duration_ms'], album=album)
                session.add(track)

        session.commit()
        logging.info(f"Data for {artist_name} successfully saved to the database")

if __name__ == "__main__":
    main()


INFO:root:Data for Taylor Swift successfully saved to the database
INFO:root:Data for Bad Bunny successfully saved to the database
INFO:root:Data for The Weeknd successfully saved to the database
INFO:root:Data for Drake successfully saved to the database
INFO:root:Data for Peso Pluma successfully saved to the database
